In [ ]:

#neural network
torch.manual_seed(12345)

# Split the training and validation sets (70% training, 15% validation)
# X_train, X_test, y_train, y_test = 
# train_test_split(df.drop("readmitted",axis=1), test_size=0.15, random_state=12345)


# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.15, random_state=12345)

X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train)
# X_val_tensor = torch.tensor(X_val.values, dtype=torch.float32)
# y_val_tensor = torch.tensor(y_val)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test)

# Fully connected neural network with one hidden layer
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        
        out = self.fc2(out)
        return out

# hyperparameters
input_size = 30  # Number of features in the input
hidden_size = 60 # Number of units in the hidden layer
num_classes = 2  # Number of output classes
num_epochs = 100
learning_rate = 0.1

# Create an instance of the neural network
model = NeuralNet(input_size, hidden_size, num_classes)

# Print the model architecture
print(model)

# Initialize the loss function
criterion = nn.CrossEntropyLoss()

# Initialize the optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X_train_tensor )
    loss = criterion(outputs, y_train_tensor)

    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print progress
    if (epoch + 1) % 1 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

# Make predictions
with torch.no_grad():
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs.data, 1)

# Calculate accuracy
accuracy = (predicted == y_test_tensor).sum().item() / len(y_test)




print(f'Test Accuracy: {accuracy:.4f}')